In [32]:
import torch
import torch.nn.functional as F 
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

dataset = Planetoid(root= '/tmp/Cora', name='Cora')

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv1 = GCNConv(16, dataset.num_classes)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index) 
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv1(x, edge_index)
        
        return F.log_softmax(x, dim = 1)
    
    model = GCN()
    optimizer = torch.optim.Adam(model.parameter(), lr = 0.01, weight_decay = 5e-4)
    data = dataset[0]
        
    model.train()
    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train.mask])
        loss.backward()
        optimizer.step()
        
    model.eval()
    _, pred = model(data).max(dim = 1)
    correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    accuracy = correct / int(data.test_mask.sum())
    print(f'Accuracy : {accuracy : .4f}')    
        
        

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\Naveen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [33]:
import networkx as nx
import random
import pandas as pd

def link_prediction_common_neighbors(graph, u, v):
    
    if v in graph[u]:
        return float('inf') 
   
    neighbors_u = set(graph[u])
    neighbors_v = set(graph[v])

    common_neighbors = neighbors_u.intersection(neighbors_v)
   
    score = len(common_neighbors)
    return score

graph = {
    'A1': ['B', 'C', 'D','A', 'B', 'E'],
    'B1': ['A', 'C'],
    'C1': ['A', 'B', 'E'],
    'D1': ['A'],
    'E1': ['C','A', 'B', 'E']
}

score = link_prediction_common_neighbors(graph, 'A1', 'E1')
print(f"Prediction Score between A and E : {score}")

Prediction Score between A and E : 4


In [34]:
import networkx as g_network
import pandas as data_handler
import rand_gen

def calculate_influence_propagation(network_graph, initial_influencers, simulation_count=10):
    def propagate_influence(network_graph, initial_influencers):

        current_active = set(initial_influencers)
        newly_current_active = set(initial_influencers)

        while newly_current_active:
            next_newly_current_active = set()

            for node in newly_current_active:
                adjacent_nodes = set(network_graph.adjacent_nodes(node))

                for neighbor in adjacent_nodes - current_active:
                    if rand_gen.rand_gen() < 0.1:  
                        next_newly_current_active.add(neighbor)
            current_active.udata_handlerate(next_newly_current_active)
            newly_current_active = next_newly_current_active

        return len(current_active)

    total_spread = 0

    for _ in range(simulation_count):
        total_spread += propagate_influence(network_graph, initial_influencers)
    
    return total_spread / simulation_count

def influence_maximization(network_graph, k):
    initial_influencers = set()

    for _ in range(k):

        best_node = None
        best_spread = 0

        for node in network_graph.nodes:

            if node not in initial_influencers:

                temp_set = initial_influencers | {node}
                spread = calculate_influence_propagation(network_graph, temp_set)

                if spread > best_spread:

                    best_spread = spread
                    best_node = node

        if best_node is not None:

            initial_influencers.add(best_node)

    return initial_influencers

    df = data_handler.read_csv('../../facebook_clean_data/tvshow_edges.csv')

network_graph = g_network.from_pandas_edgelist(df, 'nd_1', 'nd_2')

k = 3  

selected_nodes = influence_maximization(network_graph, k)
print(f"nodes selected for influence maximization: {selected_nodes}")

ModuleNotFoundError: No module named 'rand_gen'